ENUNCIADO
En el siguiente puzle infantil se pide rellenar los espacios en blanco
con números del 1 al 9 de tal modo que la ecuación sea correcta siguiendo el
orden de las operaciones. Los números no pueden repetirse. Es decir, en cada
uno de los 9 huecos en blancos deberá ir un número sin repetirse. Según
declaraciones del profesor Tran Phuong existen 362.880 combinaciones
posibles para colocar los 9 dígitos en los 9 espacios en blanco, pero sólo unas
pocas son correctas

RESOLUCION

La entrada que recibe el problema es un array que representa la ecuacion:
1)Los huecos se representan mediante 0
2)Ejemplo [0,"+",5,"=",7]
3) Los numeros caen siempre en las posiciones pares, mientras que los operadores en las posiciones impares

In [1]:
cd ..

C:\Users\aleja\Google Drive\4o\INFORMATICA\INTELIGENCIA ARTIFICIAL\PRACTICAS\PR1


In [2]:
cd ..

C:\Users\aleja\Google Drive\4o\INFORMATICA\INTELIGENCIA ARTIFICIAL\PRACTICAS


In [3]:
cd aima-python

C:\Users\aleja\Google Drive\4o\INFORMATICA\INTELIGENCIA ARTIFICIAL\PRACTICAS\aima-python


In [4]:
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [5]:
class Puzzle_Numeros (Problem):
    
    def __init__(self,ec):
        '''
        El estado es un array en el que las 9 primeras pos indican si hemos usado el numero o no.
        La siguiente es el indice por el que vamos en la ecuacion
        La penultima es el ultimo operador visitado y la ultima es el valor actual calculado.
        Necesitamos guardar el ultimo operador para poder calcular en tiempo constante el valor actual, pues imaginemos que
        tenemos 5 + _ = 7
        Primero ejecutariamos la accion avanzar, que me llevaria hasta el siguiente hueco y cuyo valor actual es 5.
        Para saber que operacion hay que llevar a cabo sobre el hueco debemos guardar el operador "+"
        '''
        self.initial = (0,0,0,0,0,0,0,0,0,0,"+",0)
        self.goal = (1,1,1,1,1,1,1,1,1,len(ec)-1,"=",ec[len(ec)-1])
        self.ecuacion=ec

    def actions(self, state):
        if(state[9]<(len(self.ecuacion)-1) and state[9]%2==0): #Quedan elementos por comprobar, y estamos en un numero
            if(self.ecuacion[state[9]]==0):
                return [("aplicar %d en hueco situado en posicion %d"%(n,state[9]),n) for n in range (1,10) if state[n-1]==0]
                #return [n for n in range (1,10) if state[n-1]==0]
            else: return [("avanzar hasta siguiente hueco",0)]
                #return [0]
        else: return []

    def result(self, state, action):
        '''
        Hay dos tipos de acciones, aquellas en las que solo tenemos que avanzar hasta el proximo hueco
        Y en las que tenemos que asignar un numero a un hueco.
        Las podemos diferenciar en el tamaño de la tupla:
                    1) ("nombre",0):Avanzar
                    2) ("nombre",numero a asignar)
        '''
        #n=action
        n=action[1]
        l=list(state)
        if n==0: #Avanzar hasta el proximo hueco
            i=l[9]
            while self.ecuacion[i]!=0 and i<(len(self.ecuacion)-1):
                #Primero operamos el elemento actual
                #Despues avanzamos al siguiente numero, guardando el operador que nos saltamos.
                if l[10]=="+":
                    l[11]+=self.ecuacion[i]
                elif l[10]=="-":
                    l[11]-=self.ecuacion[i]
                elif l[10]=="x":
                    l[11]=l[11]*self.ecuacion[i]
                elif l [10]==":":
                    l[11]=l[11]/self.ecuacion[i]
               #Nunca se va a meter en el igual, pues saldremos del bucle en la condicion state[9]=len(ec)  
                l[10]=self.ecuacion[i+1]
                i+=2
            l[9]=i  
        else: # Asignamos en el hueco el numero
            if l[10]=="+":
                l[11] += n
            elif l[10]=="-":
                l[11]-= n
            elif l[10]=="x":
                l[11]=l[11]* n
            elif l[10]==":":
                l[11]=l[11]/ n
            l[10]=self.ecuacion[l[9]+1]
            l[9]+=2
            l[n-1]=1 #Marcamos como ya usado el nmero indicado por la acción
        return tuple(l)
            
            
    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

In [6]:
p=Puzzle_Numeros([0,"+",13,"x",0,":",0,"+",0,"+",12,"x",0,"-",0,"-",11,"+",0,"x",0,":",0,"-",10,"=",66])


In [7]:
depth_first_graph_search(p).solution()

[('aplicar 9 en hueco situado en posicion 0', 9),
 ('avanzar hasta siguiente hueco', 0),
 ('aplicar 7 en hueco situado en posicion 4', 7),
 ('aplicar 2 en hueco situado en posicion 6', 2),
 ('aplicar 8 en hueco situado en posicion 8', 8),
 ('avanzar hasta siguiente hueco', 0),
 ('aplicar 4 en hueco situado en posicion 12', 4),
 ('aplicar 3 en hueco situado en posicion 14', 3),
 ('avanzar hasta siguiente hueco', 0),
 ('aplicar 6 en hueco situado en posicion 18', 6),
 ('aplicar 1 en hueco situado en posicion 20', 1),
 ('aplicar 5 en hueco situado en posicion 22', 5),
 ('avanzar hasta siguiente hueco', 0)]

In [8]:
breadth_first_graph_search(p).solution()

KeyboardInterrupt: 

In [ ]:
uniform_cost_search(p).solution()

 Tiene sentido que la busqueda en profundidad sea la mejor, pues todas las posibles soluciones están en las hojas del árbol de nodos, pues para obtener el resultado estamos obligado a colocar todos los números. De hecho, la búsqueda en profundidad va a tener que calcular más de 8! nodos para dar con la solución.

 A pesar de que nuestro árbol requiere más de 9! estados por la acción adicional de "avanzar", el cálculo del goal_test es constante; mientras que si solo llevamos los numeros situados, en una ecuacion de longitud n desconocida, el cálculo del goal_test sería lineal, lo que a la larga puede ser peor.